In [1]:
import tracemalloc

from utils_v1 import *
import traceback


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates, PAMultipleDates,
    MTMultipleDates, MSMultipleDates, PIMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential2

from src.manager.evidential import ManagerEvidential
from src.manager.multioutput import ManagerMCDropout, ManagerSingleRun, ManagerEvidential2
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.manager.evidential as evidential
from src.Logger import Logger
import pdb
import gc


Tensorflow ver. 2.6.0


In [3]:

with open('config/config.json', 'r') as f:
    config = json.load(f)
    
print("Mode: {}. Site: {}. Training: {}. Training date: {}".format(
    config['mode'], config['site'], config['training'], config['training_date']))
print(config)


Mode: evidential2. Site: PI. Training: True. Training date: current
{'training': True, 'inferring': True, 'site': 'PI', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 3, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 46, 'get_multiple_metrics': False, 'start_repetition_id': 0, 'end_repetition_id': 3, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'mask_out_non_biome': True, 'log_filename': 'log.pkl'}


In [4]:

if config['site'] == 'PA':
	if config['training_date'] == 'earlier':
		dates = [2017, 2018]
	else:
		dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	if config['training_date'] == 'earlier':
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)	

if config['mode'] == 'ensemble':
	if config['training'] == True:
		config['mode'] = 'single_run'
	else:
		sys.exit("Ensure that training is True for ensemble mode. Use infer_ensemble.ipynb for inference.")

if config['training'] == False:
	if config['site'] == 'PA':
		if config['mode'] == 'mcd' or config['mode'] == 'single_run': # MCD will do 10 inference on each of the 10 repetitions = 100 runs
			if config['training_date'] == 'current': exp = 0
			elif config['training_date'] == 'earlier': exp = 1
				
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 8; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 48; repetition_id = 0

	elif config['site'] == 'MT':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 2
			elif config['training_date'] == 'earlier': exp = 3
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 10; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 42; repetition_id = 0

	elif config['site'] == 'MS':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 40
			elif config['training_date'] == 'earlier': 
				exp = 37

		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 12; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 39; repetition_id = 0
	elif config['site'] == 'PI':

		if config['mode'] == 'mcd' or config['mode'] == 'single_run':
			if config['training_date'] == 'current': exp = 44
			elif config['training_date'] == 'earlier': exp = 27
		if config['mode'] == 'evidential2':
			if config['training_date'] == 'current': exp = 45; repetition_id = 0
			elif config['training_date'] == 'earlier': exp = 39; repetition_id = 0

else:
	exp = config['training_experiment_id']
ic(dates)


if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [5]:


if config['mode'] == 'evidential2':
    config["inference_times"] = 1
    manager_class = ManagerEvidential2
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = True
    
if config['mode'] == 'mcd':
    config["inference_times"] = 10
    manager_class = ManagerMCDropout
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = False

elif config['mode'] == 'single_run':
    config["inference_times"] = 1
    manager_class = ManagerSingleRun
    config['uncertainty_method'] = "pred_entropy_single"
    config['classes_mode'] = False



In [6]:
exp

46

In [7]:
logger = Logger()


In [8]:
if config['training'] == True:
    

    manager = manager_class(config, dataset, patchesHandler, logger)
    print("manager.config['dropout_training']", manager.config['dropout_training'])
    manager.defineExperiment(exp) # fixed
    
    manager.setExperimentPath()
    manager.createLogFolders()

    manager.loadDataset()

    # %%
    if config["training"] == True:
        manager.run()


manager.config['dropout_training'] False
D:/Jorge/datasets/deforestation/PI/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

training samples:  (7680, 3) validation samples:  (1810, 3)
time:  0


c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_enc_net (InputLayer)      [(None, 128, 128, 9) 0                                            
__________________________________________________________________________________________________
res1_net1 (Conv2D)              (None, 128, 128, 16) 1312        input_enc_net[0][0]              
__________________________________________________________________________________________________
drop_net1 (SpatialDropout2D)    (None, 128, 128, 16) 0           res1_net1[0][0]                  
__________________________________________________________________________________________________
res2_net1 (Conv2D)              (None, 128, 128, 16) 2320        drop_net1[0][0]                  
______________________________________________________________________________________________

c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


720/720 [==============================] - 35s 48ms/step - loss: 0.2788 - accuracy: 0.6214 - val_loss: 0.3060 - val_accuracy: 0.6294

Epoch 00001: val_loss did not improve from 0.29242
Early stop count: 1
Epoch: 3
Anneling Coeficient [0.2]
720/720 [==============================] - 35s 48ms/step - loss: 0.2670 - accuracy: 0.6224 - val_loss: 0.2897 - val_accuracy: 0.6288

Epoch 00001: val_loss improved from 0.29242 to 0.28970, saving model to D:/Jorge/datasets/deforestation/experiments/PI/exp46/models\resunet_0.h5
New best val loss. Val loss: 0.2897. Early stop count: 0
Epoch: 4
Anneling Coeficient [0.3]
720/720 [==============================] - 35s 48ms/step - loss: 0.2586 - accuracy: 0.6221 - val_loss: 0.2858 - val_accuracy: 0.6282

Epoch 00001: val_loss improved from 0.28970 to 0.28580, saving model to D:/Jorge/datasets/deforestation/experiments/PI/exp46/models\resunet_0.h5
New best val loss. Val loss: 0.2858. Early stop count: 0
Epoch: 5
Anneling Coeficient [0.4]
720/720 [=========

In [9]:



if config['site'] == 'PA':
	dates = [2018, 2019]
	datasetClass = PAMultipleDates
elif config['site'] == 'MT':
	dates = [2019, 2020]
	datasetClass = MTMultipleDates
elif config['site'] == 'MS':
	dates = [2019, 2020]
	datasetClass = MSMultipleDates
elif config['site'] == 'PI':
	dates = [2019, 2020]
	datasetClass = PIMultipleDates
else:
	raise ValueError('Invalid site')

dataset = datasetClass(config = config,
			   dates = dates, 
	 	addPastDeforestationInput = config['addPastDeforestationInput'],
		borderBuffer = 2)


ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerMultipleDates(
		dataset)
elif config['mode'] == 'evidential2':
	patchesHandler = PatchesHandlerEvidential2()
else:
	patchesHandler = PatchesHandler()


ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
ic| dates: [2019, 2020]
ic| self.input_image_shape: 9
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8]]


In [10]:
manager_class

src.manager.multioutput.ManagerEvidential2

In [11]:
exp

46

In [12]:
repetition_ids = manager_class(config, dataset, patchesHandler, logger).get_repetition_ids(exp)  
repetition_n = len(repetition_ids)
print("repetition_ids {}, repetition_n {}".format(repetition_ids, repetition_n))


repetition_ids [0, 1, 2], repetition_n 3


In [13]:
config['start_repetition_id']

0

In [14]:
if config['inferring'] == True:
    results = []
    error_count = 0

    # while idx < len(repetition_n):
    # for idx in range(config['start_repetition_id'], repetition_n): # 10 repetitions
    for idx in range(config['start_repetition_id'], config['end_repetition_id']): # 10 repetitions

    # for idx in range(0, 4): # 10 repetitions


        print("Beginning run number {}".format(idx))
        logger = Logger()
        manager = manager_class(config, dataset, patchesHandler, logger)

        print("manager.config", manager.config)

        manager.defineExperiment(exp) # fixed
        manager.defineRepetitionId(idx) # varying from 0 to 10
        manager.setExperimentPath()
        manager.createLogFolders()

        manager.loadDataset()

        # %%
        if config['get_multiple_metrics'] == False:
            predictor_fcn = manager.run_predictor
        else:
            predictor_fcn = manager.run_predictor_multiple_metrics
        result = predictor_fcn()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)

        with open(manager.default_log_name, 'wb') as f:
            pickle.dump(results, f)
        print("Grid execution idx: {}".format(idx))
        del manager
        gc.collect()
        # idx += 1
    

Beginning run number 0
manager.config {'training': True, 'inferring': True, 'site': 'PI', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 3, 'uncertainty_method': 'pred_entropy', 'loadInference': False, 'removePolygons': True, 'plotLandsat': False, 'plotPOI': False, 'training_experiment_id': 46, 'get_multiple_metrics': False, 'start_repetition_id': 0, 'end_repetition_id': 3, 'save_probabilities': False, 'addPastDeforestationInput': True, 'classes_mode': True, 'learning_rate': 0.0001, 'use_cloud_mask': False, 'mask_out_non_biome': True, 'log_filename': 'log.pkl', 'inference_times': 1, 'dropout_training': False}
D:/Jorge/datasets/deforestation/PI/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Label where deforestation past years is actual date (2020) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


Past deforestation before 2008 is 2
D:/Jorge/datasets/deforestation/PI/deforestation_before_2000_PI.tif


ic| np.unique(label_past_deforestation_before_2008, return_counts=True): (array([  0, 208], dtype=uint8), array([385251297,  18167940], dtype=int64))
ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2002, 2004, 2006, 2008, 2010, 2012, 2013, 2014, 2015, 2016,
                                                                    2017, 2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([340580606,   4312043,   6953620,   5659916,   7162776,   6261432,
                                                                      8191420,   3233962,   3270293,   3044419,   1817289,   2359424,
                                                                      2201926,   3050229,   2901307,   2418575], dtype=int64))


D:/Jorge/datasets/sentinel2/PI/biome_limits.tif


KeyboardInterrupt: 

In [ ]:
error_count

0

In [ ]:
print(results)

[{'uncertainty_result': {'metrics': {'precision_L': array([0.70563854]), 'recall_L': array([0.96048301]), 'recall_Ltotal': array([0.87069108]), 'AA': array([0.02999895]), 'precision_H': array([0.15253552]), 'recall_H': array([0.44877206]), 'UEO': array([0.11639467]), 'f1_L': array([0.81357069]), 'f1_H': array([0.22768274]), 'f1': 72.75}, 'metrics_audited': {'precision': array([0.72636998]), 'recall': array([0.9641773]), 'f1': array([0.8285476])}, 'exp': 44}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.56326273]), 'recall_L': array([0.95704665]), 'recall_Ltotal': array([0.86589904]), 'AA': array([0.02999832]), 'precision_H': array([0.10471903]), 'recall_H': array([0.39042686]), 'UEO': array([0.13660256]), 'f1_L': array([0.70915659]), 'f1_H': array([0.16514375]), 'f1': 62.45}, 'metrics_audited': {'precision': array([0.58874108]), 'recall': array([0.96113746]), 'f1': array([0.73020058])}, 'exp': 44}}, {'uncertainty_result': {'metrics': {'precision_L': array([0.64202204]),

In [ ]:
print(len(results))

10


In [ ]:
results[7]

{'uncertainty_result': {'metrics': {'precision_L': array([0.58988893]),
   'recall_L': array([0.98461672]),
   'recall_Ltotal': array([0.90728721]),
   'AA': array([0.02999981]),
   'precision_H': array([0.09141104]),
   'recall_H': array([0.44125552]),
   'UEO': array([0.14428742]),
   'f1_L': array([0.7377738]),
   'f1_H': array([0.15144794]),
   'f1': 64.58},
  'metrics_audited': {'precision': array([0.60981293]),
   'recall': array([0.98582489]),
   'f1': array([0.75351531])},
  'exp': 44}}